In [93]:
# !pip install pyreadstat

In [2]:
import pandas as pd
import os
from datetime import datetime
import csv
import ast
import logging
import yaml
import tqdm as tqdm
import re

# Move to the path below
os.chdir('..')
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/djrtestcompute/code/Users/damian.rumble/data_store_tools'

In [ ]:

pd.set_option('display.max_columns', None)

# load API keys
from scr.tools.load_env import load_env_file

def load_config(config_path="config.yaml"):
    """Load configuration from YAML file"""
    with open(config_path, 'r') as file:
        return yaml.safe_load(file)
    
# Load configs
config = load_config("datastore_config.yaml")

# master configs
bronze_lakehouse_name = config['datastore']['lakehouses'][1]
silver_lakehouse_name = config['datastore']['lakehouses'][0]
status = config['datastore']['status']

project_name = 'haleon_respiratory'

FABRIC_WORKSPACE_NAME= 'Haleon'
print(FABRIC_WORKSPACE_NAME)

from scr.fabric_tools import FabricTools
fabric_tools = FabricTools(project_name,FABRIC_WORKSPACE_NAME)


2026-01-30 09:56:36,108 - INFO - No environment configuration found.
2026-01-30 09:56:36,111 - INFO - ManagedIdentityCredential will use Azure ML managed identity
2026-01-30 09:56:36,115 - INFO - DefaultAzureCredential acquired a token from ManagedIdentityCredential
2026-01-30 09:56:36,118 - INFO - AzureMLCredential.get_token_info succeeded
2026-01-30 09:56:36,119 - INFO - ManagedIdentityCredential.get_token_info succeeded
2026-01-30 09:56:36,119 - INFO - DefaultAzureCredential acquired a token from ManagedIdentityCredential


Haleon
Found 4 accessible lakehouses:

Name: Diageo
Last Modified: 2025-07-18 11:22:19+00:00
--------------------------------------------------
Name: MSQ
Last Modified: 2026-01-15 12:57:55+00:00
--------------------------------------------------
Name: Haleon
Last Modified: 2026-01-20 09:12:26+00:00
--------------------------------------------------
Name: Lab
Last Modified: 2026-01-19 16:03:25+00:00
--------------------------------------------------
Operating in the Haleon workspace


In [4]:
fabric_tools.list_tables_in_lakehouse('bronze')

2026-01-30 09:56:37,242 - INFO - AzureMLCredential.get_token_info succeeded
2026-01-30 09:56:37,245 - INFO - ManagedIdentityCredential.get_token_info succeeded
2026-01-30 09:56:37,246 - INFO - DefaultAzureCredential acquired a token from ManagedIdentityCredential


['bronze_spss_data_part_1', 'dbo', 'bronze_spss_data_part_1']

In [5]:
# Pulling the taxonomy table from the datastore
table_name = "bronze_spss_data_part_1"
df_spss = fabric_tools.read_table_from_lakehouse('bronze', table_name)


table bronze.Lakehouse/Tables/dbo/bronze_spss_data_part_1 exists: True
reading from: abfss://Haleon@onelake.dfs.fabric.microsoft.com/bronze.lakehouse/Tables/dbo/bronze_spss_data_part_1


2026-01-30 09:56:39,181 - INFO - Successfully read table from abfss://Haleon@onelake.dfs.fabric.microsoft.com/bronze.lakehouse/Tables/dbo/bronze_spss_data_part_1


### Questions  

In [ ]:
'''
 SCREENER
S1. Country - Allocate automatically
S2. Industry Screen - Which industries do you or family work in
S3. Gender - What best describes your gender
S4. Age - How old are you
SQ4. NCSS - Assets owned at home (India only)
SQ5a. SEC - Chief wage earner education level (India only)
SQ5b. SEC - Chief wage earner occupation (India only)
S5. Income - Gross household income
S6. Region - Where do you live
S7. China - City - City within province (China only)
S8. India - City - City within province (India only)
S9. Area - Urban/Suburban/Rural
S10. Children - Do you have children aged under 18
S11. Age of Children - Ages of children who live with you
S12. Consent - Informed consent statements
S13. Conditions suffered - Recency of experiencing respiratory conditions
S14. Actions Taken - Actions taken to prevent/treat conditions
S14b. Chronic conditions suffered - Diagnosed chronic conditions
S15. Conditions suffered - KIDS - Recency of children's respiratory conditions
S16. ACTIONS TAKEN - KIDS - Actions taken for children's conditions
'''

In [ ]:
''' 
# ATTITUDES
A1. ATTITUDES TO RESPIRATORY HEALTH - 10-point scale attitudes (16 statements)
A2. HEALTH INCLUSIVITY - Ease of accessing health resources (4 dimensions)

# BEHAVIOURS & OCCASIONS INVENTORY
B1. L3M Symptoms Experienced by condition/issue - Symptoms for each condition
*B2. L3M Products Used by condition/issue - Products used for each condition
*B3. L3M Behaviours by CCF Illness Stage - When products used during illness stages
*B4. L3M Behaviours by Allergy Stage - When products used during allergy stages
B5. L3M Symptoms Experienced - KIDS - Children's symptoms for each condition
*B6. L3M Products Used by condition/issue - KIDS - Products used for children
*B7. L3M Behaviours by CCF Illness Stage - KIDS - When products used for children
*B8. Behaviours by Allergy Stage - KIDS - When products used for children's allergies

# BRAND FIT
C1. Brand Awareness - Brands familiar with
C2. Brand L12M Usage - Brands used in last 12 months
C3. Brand Fit - Functional - Functional associations with brands
C4. Brand Fit - Emotional - Emotional associations with brands
C5. Brand Fit - Brand/Product/Format - Product/format associations with brands

# SHOPPER
D1. Lead Shopper - Who purchases OTC medications in household
D2. Purchase Channels - Where generally purchase respiratory products
D3. Sources of Influence - Information sources used for research

# ADDITIONAL CLASSIFICATION
E1. ATTITUDES TO DEFINE the GREAT 8 - 10-point scale attitudes (8 statements)
E2. MORE ATTITUDES TO DEFINE the GREAT 8 - 10-point scale attitudes (14 statements)
E3a. Ethnicity -- USA LATINX - Hispanic/Latino identification (USA only)
E3b. Ethnicity -- USA RACE - Racial identification (USA only)
E5. Health Status - Overall health rating
E6. Health Behaviours - Regular health behaviors
E7. Frequency of contracting viral infections - How often get viral infections (CCF sample)
E8. Level of concern with developing viral infections - Concern level (CCF sample)
E9. Length of allergies - Duration of allergy experience (Allergy sample)
E10. Frequency of allergies - How often experience symptoms (Allergy sample)
E11. Recurrence of allergies - Persistent/occasional/rare (Allergy sample)
E12. Length of nasal congestion - Duration of nasal congestion (Nasal sample)
E13. Frequency of nasal congestion - How often experience symptoms (Nasal sample)
E14. Recurrence of nasal congestion - Persistent/occasional/rare (Nasal sample)
'''


In [ ]:
'''
OCASSION LOOP

Occasion loops are headed in the following ways
- L7a_1 - pressumed to be LC
- L7a_1_TB - pressumed to be LA
- L7a_1_4pt - pressumed to be LN

COMMON to ALL

L1. Description of Experience - Open-ended description of last usage occasion
L2. Allergy Experienced - Type of allergy trigger
L3. Symptoms Experienced - Specific symptoms during occasion
L4. Severity of Symptoms - How severe symptoms were
L5. Impact of Symptoms - Life aspects impacted by symptoms
L9 Brand/Product/Format Needs - L9
L11 format - L11
L12 Brand - L12
L13 Time of Day - L13
L14 Location - L14
L15 Number of times taken - L15
L16 recommendation - L16
L17 Where purchased - L17
L18 Purchase Spend - L18
L19 Other treatments and activities - L19
L20 Treatment barriers  - L20

LC -- CCF & CCF KIDS - references illness/medication type pulled from question B3/7

*LC7a. Functional Needs (scale) - L7a
*LC7b. Most Important Functional Needs (Top 5) - L7b
*LC8a. Emotional Needs (scale) - L8a
*LC8b. Most Important Emotional Needs (Top 5) - L8b
LC10 Satisfaction - L10

LA -- ALLERGY & KIDS ALLERGY - references illness/medication type pulled from question B4/8

*LA7a. Functional Needs (scale) - L7a_TB
*LA7b. Most Important Functional Needs (Top 5) - L7b_TB
*LA8a. Emotional Needs (scale) - L8a_TB
*LA8b. Most Important Emotional Needs (Top 5) - L8b_TB
LA10. Satisfaction - L10_TB

LN -- NASAL CONGESTION NOT CAUSED BY COLD/FLU/ALLERGY - references illness/medication type pulled from question B2/B9

*LN7a. Functional Needs (scale) - L7a_4pt
*LN7b. Most Important Functional Needs (Top 5) - L7b_4pt
*LN8a. Emotional Needs (scale) - L8a_4pt
*LN8b. Most Important Emotional Needs (Top 5) - L8b_4pt
LN10. Satisfaction - How well product met top needs. - MISSING???
'''

In [ ]:
''' 



'''

### Exploration

In [15]:

float_columns = []


for col in tqdm.tqdm(df_spss.select_dtypes(include=['float']).columns):
    try:
        df_spss[col] = df_spss[col].astype('Int64')
    except Exception as e:
        float_columns.append(col)
        print(f"Error converting {col}: {e}")

  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0/42 [00:00<?, ?it/s]

 52%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 22/42 [00:00<00:00, 209.39it/s]

Error converting L7a_mean1: cannot safely cast non-equivalent float64 to int64
Error converting L7a_mean2: cannot safely cast non-equivalent float64 to int64
Error converting L7a_mean3: cannot safely cast non-equivalent float64 to int64
Error converting L7a_mean4: cannot safely cast non-equivalent float64 to int64
Error converting L7b_mean1: cannot safely cast non-equivalent float64 to int64
Error converting L7b_mean2: cannot safely cast non-equivalent float64 to int64
Error converting L7b_mean3: cannot safely cast non-equivalent float64 to int64
Error converting L7b_mean4: cannot safely cast non-equivalent float64 to int64
Error converting L8a_mean1: cannot safely cast non-equivalent float64 to int64
Error converting L8a_mean2: cannot safely cast non-equivalent float64 to int64
Error converting L8a_mean3: cannot safely cast non-equivalent float64 to int64
Error converting L8a_mean4: cannot safely cast non-equivalent float64 to int64
Error converting L8b_mean1: cannot safely cast non-e

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 175.87it/s]

Error converting TherafluPRSEG2: cannot safely cast non-equivalent object to int64
Error converting TherafluPRSEG3: cannot safely cast non-equivalent object to int64
Error converting TherafluPRSEG4: cannot safely cast non-equivalent object to int64
Error converting TherafluPRSEG5: cannot safely cast non-equivalent object to int64
Error converting TherafluPRSEG6: cannot safely cast non-equivalent object to int64


In [26]:
all_columns = df_spss.columns.tolist()
all_columns

['id',
 'RecordType',
 'Ctotal',
 'Respfilter',
 'OccFilter',
 'RESPONSEID',
 'RESPID',
 'INTERVIEW_START',
 'INTERVIEW_END',
 'S1',
 'SAMPLEREC',
 'L',
 'INTRO1',
 'S2_1',
 'S2_2',
 'S2_3',
 'S2_4',
 'S2_5',
 'S2_6',
 'S2_7',
 'S2_99',
 'S3',
 'S3_4_OTHER',
 'S4',
 'AGEREC',
 'AGEGENDER',
 'SQ4_1',
 'SQ4_2',
 'SQ4_3',
 'SQ4_4',
 'SQ4_5',
 'SQ4_6',
 'SQ4_7',
 'SQ4_8',
 'SQ4_9',
 'SQ4_10',
 'SQ4_99',
 'SQ4REC',
 'SQ5A',
 'SQ5B',
 'INXSEC',
 'INXNCCS',
 'S5',
 'S6',
 'REGION',
 'S7',
 'S8',
 'S9',
 'S10',
 'S11_1',
 'S11_2',
 'S11_3',
 'S11_4',
 'S11_5',
 'S11_6',
 'S11_7',
 'S11_8',
 'S11_9',
 'S11_10',
 'S11_11',
 'S11_12',
 'S11_13',
 'S11_14',
 'S11_15',
 'S11_16',
 'S11_17',
 'S11_18',
 'S11_19',
 'S11KIDS',
 'S12_1',
 'S12_2',
 'S12_3',
 'S12_4',
 'S12_5',
 'S12_6',
 'S13_1',
 'S13_2',
 'S13_3',
 'S13_4',
 'S13_5',
 'S13_6',
 'S13_7',
 'S13_8',
 'S13_9',
 'S13_10',
 'S13_11',
 'S13_12',
 'S13_13',
 'S13_14',
 'S14_1_1',
 'S14_1_2',
 'S14_1_3',
 'S14_1_4',
 'S14_1_5',
 'S14_1_6',
 '

In [21]:
df_spss.head()

id  RecordType  Ctotal  Respfilter  OccFilter  RESPONSEID  RESPID  \
0  14501           2       1           1          1      133010  133152   
1  14501           2       1        <NA>          1      133010  133152   
2  14502           2       1           1          1      133013  133155   
3  14502           2       1        <NA>          1      133013  133155   
4  14503           2       1           1          1      133024  133166   

   INTERVIEW_START  INTERVIEW_END  S1  SAMPLEREC     L  INTRO1  S2_1  S2_2  \
0      13987569045    13987572050   2          3  2057       2     0     0   
1      13987569045    13987572050   2          3  2057       2     0     0   
2      13987569089    13987571587   7          1  1025       2     0     0   
3      13987569089    13987571587   7          1  1025       2     0     0   
4      13987569415    13987571437   3          1     4       2     0     0   

   S2_3  S2_4  S2_5  S2_6  S2_7  S2_99  S3 S3_4_OTHER  S4  AGEREC  AGEGENDER  \
0     0     0     0     0     0      1   1             30       2          2   
1     0     0     0     0     0      1   1             30       2          2   
2     0     0     0     0     0      1   2             23       1          8   
3     0     0     0     0     0      1   2             23       1          8   
4     0     0     0     0     0      1   1             28       2          2   

   SQ4_1  SQ4_2  SQ4_3  SQ4_4  SQ4_5  SQ4_6  SQ4_7  SQ4_8  SQ4_9  SQ4_10  \
0      1      1      1      1      1      1      1      1      1       1   
1      1      1      1      1      1      1      1      1      1       1   
2   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>    <NA>   
3   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>    <NA>   
4   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>    <NA>   

   SQ4_99  SQ4REC  SQ5A  SQ5B  INXSEC  INXNCCS  S5   S6  REGION    S7    S8  \
0       0       9     7     6       1        1  17   67       9  <NA>    33   
1       0       9     7     6       1        1  17   67       9  <NA>    33   
2    <NA>    <NA>  <NA>  <NA>    <NA>     <NA>  69  185    <NA>  <NA>  <NA>   
3    <NA>    <NA>  <NA>  <NA>    <NA>     <NA>  69  185    <NA>  <NA>  <NA>   
4    <NA>    <NA>  <NA>  <NA>    <NA>     <NA>  26  117      14  1959  <NA>   

   S9  S10  S11_1  S11_2  S11_3  S11_4  S11_5  S11_6  S11_7  S11_8  S11_9  \
0   1    1      0      0      0      0      0      1      0      0      0   
1   1    1      0      0      0      0      0      1      0      0      0   
2   1    3   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   
3   1    3   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   
4   1    3   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   <NA>   

   S11_10  S11_11  S11_12  S11_13  S11_14  S11_15  S11_16  S11_17  S11_18  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>   
3    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>   
4    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>    <NA>   

   S11_19  S11KIDS  S12_1  S12_2  S12_3  S12_4  S12_5  S12_6  S13_1  S13_2  \
0       0        1      1      1      1      1      1      1      3      6   
1       0        1      1      1      1      1      1      1      3      6   
2    <NA>     <NA>      1      1      1      1      1      1      1      2   
3    <NA>     <NA>      1      1      1      1      1      1      1      2   
4    <NA>     <NA>      1      1      1      1      1      1      3      6   

   S13_3  S13_4  S13_5  S13_6  S13_7  S13_8  S13_9  S13_10  S13_11  S13_12  \
0      6      6      3      6      6      6      6       6       6       6   
1      6      6      3      6      6      6      6       6       6       6   
2      6      6      3      2      4      4 

In [92]:
check_cols = get_pattern_columns_2(df_spss.columns,'L8')
check_cols

['L8a_1',
 'L8a_2',
 'L8a_3',
 'L8a_4',
 'L8a_5',
 'L8a_6',
 'L8a_7',
 'L8a_8',
 'L8a_9',
 'L8a_10',
 'L8a_11',
 'L8a_12',
 'L8a_13',
 'L8a_14',
 'L8a_15',
 'L8a_16',
 'L8a_17',
 'L8a_18',
 'L8a_19',
 'L8a_20',
 'L8b_1',
 'L8b_2',
 'L8b_3',
 'L8b_4',
 'L8b_5',
 'L8b_6',
 'L8b_7',
 'L8b_8',
 'L8b_9',
 'L8b_10',
 'L8b_11',
 'L8b_12',
 'L8b_13',
 'L8b_14',
 'L8b_15',
 'L8b_16',
 'L8b_17',
 'L8b_18',
 'L8b_19',
 'L8b_20',
 'L8a_1_TB',
 'L8a_2_TB',
 'L8a_3_TB',
 'L8a_4_TB',
 'L8a_5_TB',
 'L8a_6_TB',
 'L8a_7_TB',
 'L8a_8_TB',
 'L8a_9_TB',
 'L8a_10_TB',
 'L8a_11_TB',
 'L8a_12_TB',
 'L8a_13_TB',
 'L8a_14_TB',
 'L8a_15_TB',
 'L8a_16_TB',
 'L8a_17_TB',
 'L8a_18_TB',
 'L8a_19_TB',
 'L8a_20_TB',
 'L8a_mean1',
 'L8a_mean2',
 'L8a_mean3',
 'L8a_mean4',
 'L8b_mean1',
 'L8b_mean2',
 'L8b_mean3',
 'L8b_mean4',
 'L8a_1_4pt',
 'L8a_2_4pt',
 'L8a_3_4pt',
 'L8a_4_4pt',
 'L8a_5_4pt',
 'L8a_6_4pt',
 'L8a_7_4pt',
 'L8a_8_4pt',
 'L8a_9_4pt',
 'L8a_10_4pt',
 'L8a_11_4pt',
 'L8a_12_4pt',
 'L8a_13_4pt',
 'L8a_14_

In [97]:
df_spss[['id','Respfilter','OccFilter','cAdultOrKidOcc','cMedType','CStage','cConditionType','Index1']]

id  Respfilter  OccFilter  cAdultOrKidOcc  cMedType  CStage  \
0      14501           1          1               1         3       2   
1      14501        <NA>          1               1        12       2   
2      14502           1          1               1         5       3   
3      14502        <NA>          1               1         8       3   
4      14503           1          1               1         6       3   
...      ...         ...        ...             ...       ...     ...   
29269   5498        <NA>          1               2         7       3   
29270   5499           1          1               1         5       3   
29271   5499        <NA>          1               1         3       4   
29272   5500           1          1               1         2       9   
29273   5500        <NA>          1               2         6       3   

       cConditionType  Index1  
0                   1       1  
1                   1       2  
2                   1       1  
3                   1       2  
4                   1       1  
...               ...     ...  
29269               1       2  
29270               1       1  
29271               1       2  
29272               3       1  
29273               1       2  

[29274 rows x 8 columns]

In [55]:
df_spss.L4.value_counts()

L4
3    11381
4     9619
2     3842
5     1267
1     1035
Name: count, dtype: Int64

In [20]:
# S1. Country

countries = {
    1: "USA",
    2: "India",
    3: "China",
    4: "Brazil",
    5: "Italy",
    6: "Poland",
    7: "KSA"
}

df_spss.S1.map(countries)

0         India
1         India
2           KSA
3           KSA
4         China
          ...  
29269       KSA
29270    Brazil
29271    Brazil
29272       KSA
29273       KSA
Name: S1, Length: 29274, dtype: object

In [28]:
#  S2. Industry Screen

df_spss.S2_7.value_counts()

S2_7
0    27694
1     1580
Name: count, dtype: Int64

In [29]:
# Identify S2 columns
s2_cols = [col for col in df_spss.columns if col.startswith('S2_')]

df_spss[s2_cols].idxmax(axis=1).str.replace('S2_', '').astype(int)

0        99
1        99
2        99
3        99
4        99
         ..
29269    99
29270    99
29271    99
29272     5
29273     5
Length: 29274, dtype: int64

In [ ]:


# Extract the number after 'S2_' and get the column where value is 1
df_spss['S2'] = 

# Drop original S2 columns
df_spss = df_spss.drop(columns=s2_cols)

In [22]:
# S3. Gender
df_spss.S3_4_OTHER.value_counts()

S3_4_OTHER
                         29270
Homossexual Masculino        2
Transgender                  2
Name: count, dtype: int64

In [ ]:
# 2. Prepare Your Need State Variables
# Primary clustering inputs:

# Functional needs (LC7a/LA7a/LN7a): Importance ratings on 10-point scale
# Emotional needs (LC8a/LA8a/LN8a): Importance ratings on 10-point scale
# Context: Consider whether to include stage (prevention/initial/peak/recovery for CCF; preventative/maintenance/flare-up for allergies)

In [53]:
check_cols = get_pattern_columns(df_spss.columns,'L3_')
check_cols

['L3_1',
 'L3_2',
 'L3_3',
 'L3_4',
 'L3_5',
 'L3_6',
 'L3_7',
 'L3_8',
 'L3_9',
 'L3_10',
 'L3_11',
 'L3_12',
 'L3_13',
 'L3_14',
 'L3_15',
 'L3_16',
 'L3_17',
 'L3_18',
 'L3_19',
 'L3_20',
 'L3_21',
 'L3_22',
 'L3_23',
 'L3_24',
 'L3_25',
 'L3_26',
 'L3_27',
 'L3_28',
 'L3_29',
 'L3_30',
 'L3_98',
 'L3_99']

In [52]:
df_spss.L3

AttributeError: 'DataFrame' object has no attribute 'L3'

### pivot and map whole data set  

In [66]:
def get_grouped_pattern_columns(columns):
    """
    Returns a dictionary grouping columns by their prefix.
    
    Returns:
    --------
    dict : {prefix: [list of columns]}
    Example: {'S2': ['S2_1', 'S2_2'], 'S3': ['S3_1', 'S3_4']}
    """
    import re
    from collections import defaultdict
    
    pattern = r'^([A-Za-z0-9]+)_\d+$'
    grouped = defaultdict(list)
    
    for col in columns:
        match = re.match(pattern, col)
        if match:
            prefix = match.group(1)
            grouped[prefix].append(col)
    
    return dict(grouped)

def get_pattern_columns(columns, prefix='S2_'):
    """
    Returns columns that match the pattern: prefix followed by numbers.
    
    Parameters:
    -----------
    columns : list or Index
        List of column names from dataframe
    prefix : str
        The prefix to match (e.g., 'S2_', 'S3_', etc.)
    
    Returns:
    --------
    list : Columns matching the pattern
    """
    import re
    
    # Pattern: prefix followed by one or more digits
    pattern = f'^{re.escape(prefix)}\\d+$'
    
    return [col for col in columns if re.match(pattern, col)]

def get_pattern_columns_2(columns, prefix='S2_'):
    """
    Returns columns that start with the given prefix (any format).
    
    Parameters:
    -----------
    columns : list or Index
        List of column names from dataframe
    prefix : str
        The prefix to match (e.g., 'S2_', 'S3_', etc.)
    
    Returns:
    --------
    list : Columns starting with the prefix
    """
    return [col for col in columns if col.startswith(prefix)]


all_cols_for_melt = get_pattern_columns(df_spss.columns,'L7a')
all_cols_for_melt_2 = get_pattern_columns_2(df_spss.columns,'L7a')

# Usage:
grouped_cols = get_grouped_pattern_columns(df_spss.columns)
print(grouped_cols.keys())
# Output: {'S2': ['S2_1', 'S2_2', 'S2_3', ...], 'S3': ['S3_1', 'S3_2', ...]}


dict_keys(['S2', 'SQ4', 'S11', 'S12', 'S13', 'S14B', 'LCLSS11XKIDS', 'PREAUDXREC', 'AUDXREC', 'A1', 'A2', 'LCLSCATEGORY', 'C1', 'C2', 'LCLSC1', 'D2', 'D3', 'E1', 'E2', 'E6', 'ln2', 'L3', 'L5', 'L7a', 'L7b', 'L8a', 'L8b', 'L9', 'L10', 'L13', 'L14', 'L19', 'L20', 's15'])


In [67]:
all_cols_for_melt

[]

In [68]:
all_cols_for_melt_2

['L7a_1',
 'L7a_2',
 'L7a_3',
 'L7a_4',
 'L7a_5',
 'L7a_6',
 'L7a_7',
 'L7a_8',
 'L7a_9',
 'L7a_10',
 'L7a_11',
 'L7a_12',
 'L7a_13',
 'L7a_14',
 'L7a_15',
 'L7a_16',
 'L7a_17',
 'L7a_18',
 'L7a_19',
 'L7a_20',
 'L7a_21',
 'L7a_22',
 'L7a_23',
 'L7a_24',
 'L7a_25',
 'L7a_26',
 'L7a_27',
 'L7a_28',
 'L7a_29',
 'L7a_30',
 'L7a_1_TB',
 'L7a_2_TB',
 'L7a_3_TB',
 'L7a_4_TB',
 'L7a_5_TB',
 'L7a_6_TB',
 'L7a_7_TB',
 'L7a_8_TB',
 'L7a_9_TB',
 'L7a_10_TB',
 'L7a_11_TB',
 'L7a_12_TB',
 'L7a_13_TB',
 'L7a_14_TB',
 'L7a_15_TB',
 'L7a_16_TB',
 'L7a_17_TB',
 'L7a_18_TB',
 'L7a_19_TB',
 'L7a_20_TB',
 'L7a_21_TB',
 'L7a_22_TB',
 'L7a_23_TB',
 'L7a_24_TB',
 'L7a_25_TB',
 'L7a_26_TB',
 'L7a_27_TB',
 'L7a_28_TB',
 'L7a_29_TB',
 'L7a_30_TB',
 'L7asum',
 'L7a_mean1',
 'L7a_mean2',
 'L7a_mean3',
 'L7a_mean4',
 'L7a_1_4pt',
 'L7a_2_4pt',
 'L7a_3_4pt',
 'L7a_4_4pt',
 'L7a_5_4pt',
 'L7a_6_4pt',
 'L7a_7_4pt',
 'L7a_8_4pt',
 'L7a_9_4pt',
 'L7a_10_4pt',
 'L7a_11_4pt',
 'L7a_12_4pt',
 'L7a_13_4pt',
 'L7a_14_4pt',


In [45]:
# Remove items_to_remove from larger_list
non_melt_cols = [item for item in df_spss.columns if item not in all_cols_for_melt]
non_melt_cols

['id',
 'RecordType',
 'Ctotal',
 'Respfilter',
 'OccFilter',
 'RESPONSEID',
 'RESPID',
 'INTERVIEW_START',
 'INTERVIEW_END',
 'S1',
 'SAMPLEREC',
 'L',
 'INTRO1',
 'S3',
 'S3_4_OTHER',
 'S4',
 'AGEREC',
 'AGEGENDER',
 'SQ4_1',
 'SQ4_2',
 'SQ4_3',
 'SQ4_4',
 'SQ4_5',
 'SQ4_6',
 'SQ4_7',
 'SQ4_8',
 'SQ4_9',
 'SQ4_10',
 'SQ4_99',
 'SQ4REC',
 'SQ5A',
 'SQ5B',
 'INXSEC',
 'INXNCCS',
 'S5',
 'S6',
 'REGION',
 'S7',
 'S8',
 'S9',
 'S10',
 'S11_1',
 'S11_2',
 'S11_3',
 'S11_4',
 'S11_5',
 'S11_6',
 'S11_7',
 'S11_8',
 'S11_9',
 'S11_10',
 'S11_11',
 'S11_12',
 'S11_13',
 'S11_14',
 'S11_15',
 'S11_16',
 'S11_17',
 'S11_18',
 'S11_19',
 'S11KIDS',
 'S12_1',
 'S12_2',
 'S12_3',
 'S12_4',
 'S12_5',
 'S12_6',
 'S13_1',
 'S13_2',
 'S13_3',
 'S13_4',
 'S13_5',
 'S13_6',
 'S13_7',
 'S13_8',
 'S13_9',
 'S13_10',
 'S13_11',
 'S13_12',
 'S13_13',
 'S13_14',
 'S14_1_1',
 'S14_1_2',
 'S14_1_3',
 'S14_1_4',
 'S14_1_5',
 'S14_1_6',
 'S14_1_7',
 'S14_1_8',
 'S14_1_9',
 'S14_1_10',
 'S14_1_11',
 'S14_1_12',
